In [1]:
import pandas as pd
from datetime import datetime
from functools import reduce
import os
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from datetime import datetime, timedelta

In [2]:
end_date = '2024-04-01'
end_date = datetime.strptime(end_date, '%Y-%m-%d')  # Convert end_date to datetime object
new_date = end_date - timedelta(days=1)  # Subtract 1 day from end_date

new_date_str = new_date.strftime('%Y-%m-%d')  # Convert new_date back to string format
print(new_date_str)  

2024-03-31


In [3]:
# Load environment variables from .env file
load_dotenv()

# Import environment variables
start_date = os.getenv('start_date')
end_date = os.getenv('end_date')

In [4]:
# Create a dummy DataFrame with the date range
dummy_df = pd.DataFrame({'Date': pd.date_range(start=start_date, end=new_date_str)})
dummy_df['Date'] = pd.to_datetime(dummy_df['Date']).dt.strftime("%Y-%m-%d")
dummy_df['Date'] = dummy_df['Date'].astype('datetime64[ns]')

## Yahoo Finance

In [5]:
df_yahoo = pd.read_parquet('C:\\Users\\Stamatis\\Desktop\\MLCryptoPredictor\\MLCryptoPredictor\\data\\external\\Crypto_Historical_Prices\\files\\yahoo_finance.parquet.gzip')

# Check for missing values

# df_yahoo.isna().sum() 

# Fill missing values with the previous value and interpolate the rest
df_yahoo = df_yahoo.bfill().interpolate(method='linear')

merged_df_1 = pd.merge(dummy_df, df_yahoo, on='Date', how='left')

# plt.plot(df_yahoo.Date, df_yahoo['GOLD_ADJ_CLOSE'], label='GOLD_ADJ_CLOSE')

## Fear and Greed Index

In [6]:
df_fear = pd.read_parquet('C:\\Users\\Stamatis\\Desktop\\MLCryptoPredictor\\MLCryptoPredictor\\data\\external\\Fear_and_Greed_Index\\files\\btc_fear_and_greed.parquet.gzip')

df_fear = df_fear[['timestamp', 'value', 'value_classification']]
df_fear.rename(columns={'timestamp': 'Date'}, inplace=True)
df_fear.sort_values(by='Date', inplace=True)
df_fear.reset_index(drop=True, inplace=True)    

# Check for missing values
df_fear.isna().sum()

merged_df_2 = pd.merge(dummy_df, df_fear, on='Date', how='left')

# Fill missing values with the previous value and interpolate the rest
merged_df_2 = merged_df_2.bfill().interpolate(method='linear')

df = pd.DataFrame(merged_df_2)

# Pivot the dataframe to get counts of each value classification for each date
pivot_df = df.pivot_table(index='Date', columns='value_classification', aggfunc='size', fill_value=0)

# Join the pivot table with the original dataframe
df = df.merge(pivot_df, on='Date')

# Rename the columns
df.columns = df.columns.str.replace(' ', '_')  # Replace spaces with underscores

C:\Users\Stamatis\AppData\Local\Temp\ipykernel_107176\3900876493.py:14: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  merged_df_2 = merged_df_2.bfill().interpolate(method='linear')


## FRED

In [7]:
df_fred = pd.read_parquet('C:\\Users\\Stamatis\\Desktop\\MLCryptoPredictor\\MLCryptoPredictor\\data\\external\\FRED\\files\\fred_processed.parquet.gzip')

df_fred.rename(columns={'index': 'Date'}, inplace=True)
df_fred.sort_values(by='Date', inplace=True)
df_fred.reset_index(drop=True, inplace=True) 

# Check for missing values
df_fred.isna().sum()

merged_df_3 = pd.merge(dummy_df, df_fred, on='Date', how='left')

## Technical Indicators

In [8]:
df_tech_ind = pd.read_parquet('C:\\Users\\Stamatis\\Desktop\\MLCryptoPredictor\\MLCryptoPredictor\\data\\external\\Technical_Indicators\\files\\btc_technical_indicators.parquet.gzip')

# Check for missing values
df_tech_ind.isna().sum()

merged_df_4 = pd.merge(dummy_df, df_tech_ind, on='Date', how='left')

# Fill missing values with the previous value and interpolate the rest
merged_df_4 = merged_df_4.bfill().interpolate(method='linear')

## Social Media - Aggregations (Twitter)

In [9]:
df_tweet_grouped = pd.read_parquet('C:\\Users\\Stamatis\\Desktop\\MLCryptoPredictor\\MLCryptoPredictor\\data\\external\\Social_Media\\files\\Bitcoin_tweets_grouped.parquet.gzip')

df_tweet_grouped.rename(columns={'date': 'Date'}, inplace=True)

df_tweet_grouped['Date'] = pd.to_datetime(df_tweet_grouped['Date'])

merged_df_5 = pd.merge(dummy_df, df_tweet_grouped, on='Date', how='left')

# Fill missing values with the previous value and interpolate the rest
merged_df_5 = merged_df_5.bfill().interpolate(method='linear')

In [10]:
# plt.plot(merged_df_5.Date, merged_df_5['tweet_count'], label='tweet_count')

## Static CSV Data

In [11]:
df_raw = pd.read_parquet('C:\\Users\\Stamatis\\Desktop\\MLCryptoPredictor\\MLCryptoPredictor\\data\\raw\\files\\raw_data.parquet.gzip')

df_raw['Date'] = df_raw['Date'].astype('datetime64[ns]')

# Check for missing values
df_raw.isna().sum()

merged_df_6 = pd.merge(dummy_df, df_raw, on='Date', how='left')

# Fill missing values with the previous value and interpolate the rest
merged_df_6 = merged_df_6.bfill().interpolate(method='linear')

C:\Users\Stamatis\AppData\Local\Temp\ipykernel_107176\1517533552.py:11: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  merged_df_6 = merged_df_6.bfill().interpolate(method='linear')


## Target Creation (Y variable)

In [12]:
df_target = pd.read_parquet('C:\\Users\\Stamatis\\Desktop\\MLCryptoPredictor\\MLCryptoPredictor\\data\\processed\\files\\btc_target.parquet.gzip')

df_target['Date'] = df_target['Date'].astype('datetime64[ns]')

# Check for missing values
df_raw.isna().sum()

merged_df_7 = pd.merge(dummy_df, df_target, on='Date', how='left')

# Fill missing values with the previous value and interpolate the rest
merged_df_7 = merged_df_7.bfill().interpolate(method='linear')

# Merge all files

In [13]:
df_processed = pd.concat([merged_df_1, df, merged_df_3, merged_df_4, merged_df_5, merged_df_6, merged_df_7], axis=1)

# Remove duplicate columns
df_processed = df_processed.loc[:, ~df_processed.columns.duplicated()]

columns_to_delete = ['BTC_OPEN', 'BTC_HIGH', 'BTC_LOW', 'BTC_CLOSE', 'BTC_ADJ_CLOSE', 'Adj_Close']

# Delete the specified columns
df_processed = df_processed.drop(columns=columns_to_delete)

df_processed.rename(columns={'Market cap': 'MARKET_CAP', 'Volume (24h)' : 'CRYPTO_VOLUME_24' }, inplace=True)

del df_processed['var_name'], df_processed['var_label']

# Columns to move to the beginning
columns_to_move = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']

# Reorder columns
df_processed = df_processed[columns_to_move + [col for col in df_processed if col not in columns_to_move]]

In [14]:
print(df_processed.columns.to_list())

['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'GOLD_ADJ_CLOSE', 'SILVER_ADJ_CLOSE', 'OIL_ADJ_CLOSE', 'GOLD_VOLUME', 'SILVER_VOLUME', 'OIL_VOLUME', 'EUR_USD_ADJ_CLOSE', 'USD_JPY_ADJ_CLOSE', 'GBP_USD_ADJ_CLOSE', 'USD_CNY_ADJ_CLOSE', 'VIX_ADJ_CLOSE', 'CBOE_INTEREST_RATE_ADJ_CLOSE', 'TREASURY_YIELD_5YRS_ADJ_CLOSE', 'RUSSEL_2000_ADJ_CLOSE', 'ISHARES_20YR_ADJ_CLOSE', 'TREASURY_BILL_13WK_ADJ_CLOSE', 'RUSSEL_2000_VOLUME', 'ISHARES_20YR_VOLUME', 'TESLA_ADJ_CLOSE', 'AMD_ADJ_CLOSE', 'INTEL_ADJ_CLOSE', 'APPLE_ADJ_CLOSE', 'NVIDIA_ADJ_CLOSE', 'META_ADJ_CLOSE', 'GOOGLE_ADJ_CLOSE', 'TESLA_VOLUME', 'AMD_VOLUME', 'INTEL_VOLUME', 'APPLE_VOLUME', 'NVIDIA_VOLUME', 'META_VOLUME', 'GOOGLE_VOLUME', 'GBTC_ADJ_CLOSE', 'ARKB_ADJ_CLOSE', 'BITB_ADJ_CLOSE', 'FBTC_ADJ_CLOSE', 'BTCO_ADJ_CLOSE', 'IBIT_ADJ_CLOSE', 'HODL_ADJ_CLOSE', 'BITO_ADJ_CLOSE', 'GBTC_VOLUME', 'ARKB_VOLUME', 'BITB_VOLUME', 'FBTC_VOLUME', 'BTCO_VOLUME', 'IBIT_VOLUME', 'HODL_VOLUME', 'BITO_VOLUME', 'BTC_VOLUME', 'ETH_ADJ_CLOSE', 'ET

In [49]:
len(df_processed.columns)
# print(df_processed.columns.tolist())

312

In [51]:
df_processed.to_parquet('C:\\Users\\Stamatis\\Desktop\\MLCryptoPredictor\\MLCryptoPredictor\\data\\processed\\files\\processed_data.parquet.gzip', compression='gzip')